#### Analytics on the reviews

In [2]:
import psycopg2
import pandas as pd
import torch
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector
from tqdm import tqdm
import matplotlib.pyplot as plt

In [3]:
# connecting to a post gres instance
conn = psycopg2.connect(
    host='localhost',
    user='postgres',
    port=5439,
    database='postgres',
    password='ManiAnu@12345'
)
conn.readonly = False
conn.autocommit = True



cursor = conn.cursor(cursor_factory = psycopg2.extras.RealDictCursor)
register_vector(cursor)

In [4]:
# loading the mode
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

C:\Users\manik\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\utils\generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [58]:
# loading the data to pandas df
reviews = pd.read_sql("""select * from reviews;""", conn)

C:\Users\manik\AppData\Local\Temp\ipykernel_11380\610105620.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  reviews = pd.read_sql("""select * from reviews;""", conn)


In [20]:
# generating the embedding for the following sentence
liked_reviews = ["I Loved this book "]
liked_reviews = model.encode(liked_reviews)

In [21]:
liked_embed = liked_reviews[0].tolist()
print(len(liked_embed))

384


In [37]:
#query to get the reviews that are like by a user
liked_query = f"""
SELECT reviewerid, asin, reviewtext, (embedding <=>'{liked_embed}') AS cos_distance
FROM reviews
WHERE (embedding <=>'{liked_embed}') < 0.4
ORDER BY (embedding <=> '{liked_embed}') ASC;
;
"""

cursor.execute(liked_query)
liked_results = pd.DataFrame(cursor.fetchall())


In [45]:
liked_results

,reviewerid,asin,reviewtext,cos_distance
0,A39TZ8P440NAWL,B00GWNJK14,Loved this book and characters.,0.000000
1,A1GTS3YLE3UMNL,B00BV0IM9E,Loved this book and the characters in it. Char...,0.127985
2,A1GTS3YLE3UMNL,B00D002DE8,This book was written very well. I enjoyed it ...,0.144478
3,A1ANG1191JS7J1,B009TU9M06,I loved this book. This was a great book. I lo...,0.145293
4,ABVXY3D3CCY1Q,B00J0KYFSO,I loved this book! It was such a great follow ...,0.147309
...,...,...,...,...
12412,A2IEBRCRJ5YROH,B00HGOMTP2,I'm so loved this book...I'm glad none of my f...,0.399995
12413,A958LJ1BUJLG5,B00BF4FXK2,You could read this book before the other two ...,0.399996
12414,A10Y597K071WTQ,B00D3XWOUK,"The writing is fantastic in this book, it was ...",0.399998
12415,A26KWG162U8VBQ,B00CJV1DVS,I absolutely LOVE this series. Fun and funny ...,0.399999


In [56]:
# top 5 Liked books
liked_results["asin"].value_counts().to_frame()[:5]

,count
asin,
B00FGFYIS4,46
B00BTN2DSY,46
B00BTIDW4S,44
B00BT0J8ZS,35
B00GA644CI,31


In [49]:
# generating embedding for slow read
want_to_read_this_books = ["slow read"]
wanted_to_read_this_embed = model.encode(want_to_read_this_books)
wanted_to_read_this_embed = wanted_to_read_this_embed[0].tolist()

In [50]:
# those records with slow read
wanted_to_read_this_query = f"""
SELECT reviewerid, asin, reviewtext, (embedding <=>'{wanted_to_read_this_embed}') AS cos_distance
FROM reviews

ORDER BY (embedding <=> '{wanted_to_read_this_embed}') ASC;
;
"""

cursor.execute(wanted_to_read_this_query)
wanted_to_read_this_query_results = pd.DataFrame(cursor.fetchall())


In [52]:
wanted_to_read_this_query_results

,reviewerid,asin,reviewtext,cos_distance
0,A3Q23RA7DPGNG5,B00FF4IAGM,"Good read was slow at first can,t wait for boo...",0.310311
1,A21ZNOB5DP8WH5,B00JTKG5YC,Slow read but picked up at the end will read p...,0.318622
2,A31Z94E63H1N46,B0052LFTOO,a good read for adults who are in to the serie...,0.362775
3,AD6KB2AZ7W1P7,B004ZCIG64,Started slow but picked up fast. Good read wit...,0.363133
4,A6SR7U2PNGN0N,B005OC5QXK,The flow of the book is slow for me. I liked t...,0.370008
...,...,...,...,...
102355,A8A4LKHUV6QB0,B007UC0YWA,"When the title says ""Dallas Billionaires"", I e...",1.204600
102356,AN9O9JE1B8LLO,B0064PJUWY,"Ahhhhh the background was engaging, so I am gu...",1.206660
102357,A2LCCVPM8QKOBM,B00DMNDTR8,I don't know what to say. Amir is the truth. I...,1.217825
102358,A3DQEUN783MVJ,B00GYLAZJU,If I could run away from and ex into the arms ...,1.220583


### Recommendation System

In [100]:
recommendation_query_similar_users = """

WITH ReviewedItems AS (
    SELECT asin, embedding, reviewtext
    FROM reviews
    WHERE reviewerid = 'A387F7NOAY4ERC'
)
SELECT
    'A387F7NOAY4ERC' AS initial_reviewer, 
    r2.reviewerid AS other_reviewer,
    r1.asin,
    r1.reviewtext AS initial_review,
    r2.reviewtext AS other_review,
    (r1.embedding <=> r2.embedding) AS cosine_distance
FROM
    ReviewedItems r1
JOIN
    reviews r2 ON r1.asin = r2.asin AND r2.reviewerid != 'A387F7NOAY4ERC'
ORDER BY
    r1.asin, cosine_distance ASC;
"""

In [106]:
def fetch_similar_users(reviewer_id):
    """
    Fetches similar users for a specified reviewer based on cosine distances of 
    embeddings with other user reviews of common items.

    Parameters:
       
        reviewer_id (str): The ID of the reviewer for whom to fetch comparisons.

    Returns:
        DataFrame: A pandas DataFrame containing the comparison results.
    """

    # Establish a connection to the PostgreSQL database


    # SQL query to fetch review comparisons
    recommendation_query_similar_users = """

    WITH ReviewedItems AS (
        SELECT asin, embedding, reviewtext
        FROM reviews
        WHERE reviewerid = %s
    )
    SELECT
        %s AS initial_reviewer, 
        r2.reviewerid AS other_reviewer,
        r1.asin,
        r1.reviewtext AS initial_review,
        r2.reviewtext AS other_review,
        (r1.embedding <=> r2.embedding) AS cosine_distance
    FROM
        ReviewedItems r1
    JOIN
        reviews r2 ON r1.asin = r2.asin AND r2.reviewerid != %s
    ORDER BY
        r1.asin, cosine_distance ASC;
    """
    cursor.execute(recommendation_query_similar_users, (reviewer_id, reviewer_id, reviewer_id))
    results = cursor.fetchall()

    # Convert results to a pandas DataFrame
    recommended_data = pd.DataFrame(results, columns=['initial_reviewer', 'other_reviewer', 'asin', 'initial_review', 'other_review', 'cosine_distance'])
    print(recommended_data.head())
    similar_users = recommended_data[recommended_data["cosine_distance"]<0.4][["initial_reviewer","other_reviewer"]].value_counts().to_frame().reset_index()["other_reviewer"].values[:2]

    return similar_users


In [110]:
reviewer_id = 'A387F7NOAY4ERC'
similar_users = fetch_similar_users(reviewer_id)
print("The similar users for {} : ".format(reviewer_id), similar_users)

  initial_reviewer  other_reviewer        asin  \
0   A387F7NOAY4ERC   ALFZHX7BF8LTP  B004VWMMUO   
1   A387F7NOAY4ERC  A2U72WDAUX5CD8  B004VWMMUO   
2   A387F7NOAY4ERC  A3JSO74SPEM1PL  B004VWMMUO   
3   A387F7NOAY4ERC  A223QCN8I8R2KK  B004VWMMUO   
4   A387F7NOAY4ERC   ALYN2W1QZ3PXI  B004VWMMUO   

                                      initial_review  \
0  5 plus stars!!  I loved the genie he was just ...   
1  5 plus stars!!  I loved the genie he was just ...   
2  5 plus stars!!  I loved the genie he was just ...   
3  5 plus stars!!  I loved the genie he was just ...   
4  5 plus stars!!  I loved the genie he was just ...   

                                        other_review  cosine_distance  
0  I love Kristen Ashley books but have always pu...         0.324733  
1  Not usually a fan of genie or paranormal stori...         0.342222  
2  Wow, wow, wow another fantastic read from Kris...         0.367272  
3  I'll be the first to admit that this just didn...         0.400174  
4 

In [128]:
similar_users = recommended_data[recommended_data["cosine_distance"]<0.4][["initial_reviewer","other_reviewer"]].value_counts().to_frame().reset_index()["other_reviewer"].values[:2].tolist()

In [144]:
like = ["Loved it!"]
embedding_for_liked = model.encode(like)
embedding_for_liked = embedding_for_liked[0].tolist()

In [157]:
def generate_recommendation(reviewer_id, similar_users):
    """
    Fetches recommendation for a specified reviewer based on cosine distances of 
    embeddings Loved it

    Parameters:
       
        reviewer_id (str): The ID of the reviewer for whom to fetch comparisons.
        similar_users    : The list of similar users

    Returns:
        DataFrame: A pandas DataFrame containing the comparison results.
    """



    # SQL query to fetch recommendations
    recommendation_query = f"""
    WITH UserReviewed AS (
        SELECT DISTINCT asin
        FROM reviews
        WHERE reviewerid = %s
    ),
    SimilarUsersReviews AS (
        SELECT asin, embedding
        FROM reviews
        WHERE reviewerid = ANY(%s) 
    )
    SELECT sur.asin
    FROM SimilarUsersReviews sur
    WHERE sur.asin NOT IN (SELECT asin FROM UserReviewed)
      AND (sur.embedding <=> '{embedding_for_liked}') < 0.4  
    ORDER BY (sur.embedding <=> '{embedding_for_liked}') ASC;
    """
    cursor.execute(recommendation_query, (reviewer_id, similar_users))
    results = cursor.fetchall()

    # Convert results to a pandas DataFrame
    recommended_items = pd.DataFrame(results, columns=['asin'])
    # print(recommended_items.head())
    return recommended_items['asin'].values


In [158]:
recommendations = generate_recommendation('A387F7NOAY4ERC', similar_users)
print("Recommendation for the user {}: ".format(reviewer_id), recommendations)

Recommendation for the user A387F7NOAY4ERC:  ['B00BD2PT8M']


In [ ]:
pro